In [ ]:
import os
PROJECT_FOLDER = os.path.join(os.path.dirname(os.getcwd()), "storage")
os.chdir(PROJECT_FOLDER)
print(os.getcwd())

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import keras
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import Adam, RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

Using TensorFlow backend.


In [4]:
train_path = os.path.join("dogs-vs-cats(smaller)", "train")
valid_path = os.path.join("dogs-vs-cats(smaller)", "valid")
test_path = os.path.join("dogs-vs-cats(smaller)", "test")

In [ ]:
train_batches = ImageDataGenerator().flow_from_directory(train_path,
                                                        target_size=(224, 224),
                                                        classes=["dogs", "cats"], # define which is 0, 1
                                                        batch_size=64)

valid_batches = ImageDataGenerator().flow_from_directory(valid_path,
                                                        target_size=(224, 224),
                                                        classes=["dogs", "cats"],
                                                        batch_size=4)

test_batches = ImageDataGenerator().flow_from_directory(test_path,
                                                        target_size=(224, 224),
                                                        classes=["dogs", "cats"],
                                                        batch_size=10)

build model

In [8]:
vgg16_model = keras.applications.vgg16.VGG16()

W0726 18:55:01.038536 17768 deprecation_wrapper.py:119] From D:\virtualenvs\congyuml\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0726 18:55:01.052497 17768 deprecation_wrapper.py:119] From D:\virtualenvs\congyuml\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0726 18:55:01.057488 17768 deprecation_wrapper.py:119] From D:\virtualenvs\congyuml\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0726 18:55:01.088403 17768 deprecation_wrapper.py:119] From D:\virtualenvs\congyuml\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0726 18:55:02.838014 17768 deprecation_wrapper.py:119] From D:\virtualenvs\congyum

In [24]:
print(type(vgg16_model))

model=Sequential()
for layer in vgg16_model.layers[:-1]:
    model.add(layer)

print(type(model))

<class 'keras.engine.training.Model'>
<class 'keras.engine.sequential.Sequential'>


In [25]:
print(len(model.layers))
model.layers.pop()  # this does not work anymore
print(len(model.layers))

21
21


In [26]:
for layer in model.layers:
    layer.trainable = False
model.add(Dense(2, activation='softmax'))

In [27]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       295168    
__________

In [28]:
model.compile(Adam(lr=.0001),
              loss="categorical_crossentropy",
              metrics=["accuracy"]
             )

model.fit_generator(train_batches,
                    epochs=3,
                    steps_per_epoch=8,
                    
                    validation_data=valid_batches,
                    validation_steps=4)

Epoch 1/6
8/8 [==============================] - 30s 4s/step - loss: 0.5850 - acc: 0.7264 - val_loss: 1.9578 - val_acc: 0.6875
Epoch 2/6
8/8 [==============================] - 33s 4s/step - loss: 0.0463 - acc: 0.9961 - val_loss: 3.1371 - val_acc: 0.5000
Epoch 3/6
8/8 [==============================] - 32s 4s/step - loss: 0.0156 - acc: 1.0000 - val_loss: 4.4185 - val_acc: 0.4375
Epoch 4/6


KeyboardInterrupt: 

In [ ]:
# experience: improve batch size to enable learning.
# improve number of parameters to let it actually learn some thing